In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
print(res)

soup = BeautifulSoup(res.text,'lxml') #, 'html.parser')
tr = soup.find_all('tr')
# tr.find('td')

<Response [200]>


In [3]:
table = soup.find('table',style='width:100%; border-collapse:collapse; border:1px solid #ccc;')

In [4]:
#extract tr format from the page:
td = table.find_all('tr')

# create dataframe with columns names:
df = pd.DataFrame(columns = ['Postalcode','Borough','Neighborhood'])

# extract td format so we can extract the information we want:
for i in td:
    for td in i.find_all('td'):
        
        postel_code = ''
        nigh = ''
        borough = ''
        
        
        # Extracting Postalcode
        text = td.text.strip()
        postel_code = text[0:3]
        
        
        # Extracting Borough
        text = text[3:]
        borough = text.split('(')[0]
        
        
        # Cleaning Text to extract Neighborhood 
        try:
            nigh = text.split('(')[1]
            nigh = nigh.replace(')','')
            nigh = nigh.split('/')
            nigh = [i.strip() for i in nigh]
            nigh = ', '.join(nigh)
        except:
            continue
        
        
        if borough != 'Not assigned':
            df = df.append({'Postalcode': postel_code,'Borough': borough,'Neighborhood': nigh} ,ignore_index=True)


In [8]:
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M9A,Etobicoke,Islington Avenue


In [6]:
df.shape

(102, 3)

In [11]:
df.shape[0]

102

# geocoder Example won't work :(

In [59]:
# import geocoder # import geocoder


# def get_geocoder(pc):
#     # initialize your variable to None
#     lat_lng_coords = None

#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(pc))
#         lat_lng_coords = g.latlng

#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]
#     print(latitude,longitude)
#     return latitude,longitude

In [61]:
# for i in range(0,len(df)):
#     df['lat'][i],df['long'][i]=get_geocoder(df.iloc[i]['Postalcode'])

In [62]:
cord = pd.read_csv('Geospatial_Coordinates.csv')

In [66]:
df = df.join(cord)

In [68]:
df = df[['Postalcode', 'Borough', 'Neighborhood', 'Latitude',
       'Longitude']]

In [69]:
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M9A,Etobicoke,Islington Avenue,43.773136,-79.239476
